<a href="https://colab.research.google.com/github/seawavve/PeekABook/blob/main/Spark_NER_trimmed.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget http://setup.johnsnowlabs.com/colab.sh -O - | bash

--2021-07-08 06:32:48--  http://setup.johnsnowlabs.com/colab.sh
Resolving setup.johnsnowlabs.com (setup.johnsnowlabs.com)... 51.158.130.125
Connecting to setup.johnsnowlabs.com (setup.johnsnowlabs.com)|51.158.130.125|:80... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp/master/scripts/colab_setup.sh [following]
--2021-07-08 06:32:48--  https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp/master/scripts/colab_setup.sh
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1608 (1.6K) [text/plain]
Saving to: ‘STDOUT’

-                   100%[===================>]   1.57K  --.-KB/s    in 0s      

2021-07-08 06:32:49 (28.4 MB/s) - written to stdout [1608/1608]

setup Cola

In [2]:
import json
import os
from pyspark.ml import Pipeline
from sparknlp.base import *
from sparknlp.annotator import *
import sparknlp

spark = sparknlp.start()

def get_ann_pipeline ():
    
    document_assembler = DocumentAssembler() \
        .setInputCol("text")\
        .setOutputCol('document')

    sentence = SentenceDetector()\
        .setInputCols(['document'])\
        .setOutputCol('sentence')\
        .setCustomBounds(['\n'])

    tokenizer = Tokenizer() \
        .setInputCols(["sentence"]) \
        .setOutputCol("token")

    pos = PerceptronModel.pretrained() \
              .setInputCols(["sentence", "token"]) \
              .setOutputCol("pos")
    
    embeddings = WordEmbeddingsModel.pretrained()\
          .setInputCols(["sentence", "token"])\
          .setOutputCol("embeddings")

    ner_model = NerDLModel.pretrained() \
          .setInputCols(["sentence", "token", "embeddings"]) \
          .setOutputCol("ner")

    ner_converter = NerConverter()\
      .setInputCols(["sentence", "token", "ner"])\
      .setOutputCol("ner_chunk")

    ner_pipeline = Pipeline(
        stages = [
            document_assembler,
            sentence,
            tokenizer,
            pos,
            embeddings,
            ner_model,
            ner_converter
        ]
    )

    empty_data = spark.createDataFrame([[""]]).toDF("text")

    ner_pipelineFit = ner_pipeline.fit(empty_data)

    ner_lp_pipeline = LightPipeline(ner_pipelineFit)

    print ("Spark NLP NER lightpipeline is created")

    return ner_lp_pipeline


In [3]:
conll_pipeline = get_ann_pipeline ()

pos_anc download started this may take some time.
Approximate size to download 3.9 MB
[OK!]
glove_100d download started this may take some time.
Approximate size to download 145.3 MB
[OK!]
ner_dl download started this may take some time.
Approximate size to download 13.6 MB
[OK!]
Spark NLP NER lightpipeline is created


In [6]:
parsed = conll_pipeline.annotate ("Peter Parker Baker is in a baby blue Cadillac.")

for key in parsed.keys():
    print(key,': ',parsed[key])
    
#parsed

document :  ['Peter Parker Baker is in a baby blue Cadillac.']
ner_chunk :  ['Peter Parker Baker', 'Cadillac']
pos :  ['NNP', 'NNP', 'NNP', 'VBZ', 'IN', 'DT', 'NN', 'JJ', 'NNP', '.']
token :  ['Peter', 'Parker', 'Baker', 'is', 'in', 'a', 'baby', 'blue', 'Cadillac', '.']
ner :  ['B-PER', 'I-PER', 'I-PER', 'O', 'O', 'O', 'O', 'O', 'B-ORG', 'O']
embeddings :  ['Peter', 'Parker', 'Baker', 'is', 'in', 'a', 'baby', 'blue', 'Cadillac', '.']
sentence :  ['Peter Parker Baker is in a baby blue Cadillac.']


In [5]:
conll_lines=''

for token, pos, ner in zip(parsed['token'],parsed['pos'],parsed['ner']):

    conll_lines += "{} {} {} {}\n".format(token, pos, pos, ner)


print(conll_lines)

Peter NNP NNP B-PER
Parker NNP NNP I-PER
is VBZ VBZ O
in IN IN O
a DT DT O
baby NN NN O
blue JJ JJ O
Cadillac NNP NNP B-ORG
. . . O



In [14]:
# csv로 자동화
data=[]
sentences=['Peter Parker Baker is in a baby blue Cadillac.','I love you.']
for sentence in sentences:
  parsed = conll_pipeline.annotate (sentence)
  if 'B-PER' in parsed['ner']:
    name_list=[]
    #PER가진애들 집어넣기
  else:
    data.append([])
print(data)

[['Peter'], ['Parker'], ['Baker'], ['Baker'], ['Baker'], ['Baker'], ['Baker'], ['Baker'], ['Baker'], ['Baker'], []]
